In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import os
import torch
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


# submission

In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('mmdetection/configs/yolox/yolox_x.py')

root='../dataset/'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'


#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.pipeline[1]['img_scale'] = (640,640) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/inseo_test/YOLOX_X(psuedo label)_3'
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [6]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
epoch = 'best_bbox_mAP_50_epoch_26'

In [8]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
model.eval()

YOLOX(
  (backbone): CSPDarknet(
    (stem): Focus(
      (conv): ConvModule(
        (conv): Conv2d(12, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
    )
    (stage1): Sequential(
      (0): ConvModule(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
      (1): CSPLayer(
        (main_conv): ConvModule(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (activate): Swish()
        )
        (short_conv): ConvModule(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(

In [9]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print('number of param :',pytorch_total_params)

number of param : 99004045


In [11]:
checkpoint = torch.load(checkpoint_path)['state_dict']

In [12]:
model.load_state_dict(checkpoint,strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['ema_backbone_stem_conv_conv_weight', 'ema_backbone_stem_conv_bn_weight', 'ema_backbone_stem_conv_bn_bias', 'ema_backbone_stem_conv_bn_running_mean', 'ema_backbone_stem_conv_bn_running_var', 'ema_backbone_stem_conv_bn_num_batches_tracked', 'ema_backbone_stage1_0_conv_weight', 'ema_backbone_stage1_0_bn_weight', 'ema_backbone_stage1_0_bn_bias', 'ema_backbone_stage1_0_bn_running_mean', 'ema_backbone_stage1_0_bn_running_var', 'ema_backbone_stage1_0_bn_num_batches_tracked', 'ema_backbone_stage1_1_main_conv_conv_weight', 'ema_backbone_stage1_1_main_conv_bn_weight', 'ema_backbone_stage1_1_main_conv_bn_bias', 'ema_backbone_stage1_1_main_conv_bn_running_mean', 'ema_backbone_stage1_1_main_conv_bn_running_var', 'ema_backbone_stage1_1_main_conv_bn_num_batches_tracked', 'ema_backbone_stage1_1_short_conv_conv_weight', 'ema_backbone_stage1_1_short_conv_bn_weight', 'ema_backbone_stage1_1_short_conv_bn_bias', 'ema_backbone_stage1_1_short_conv_bn_runni

In [13]:
cfg.work_dir

'./work_dirs/inseo_test/YOLOX_X(psuedo label)_3'

In [14]:
# checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [15]:
data_loader

In [16]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 21.2 task/s, elapsed: 229s, ETA:     0s

In [17]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_TTA_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


PredictionString       image_id
0  0 0.08367297 213.47339 691.88116 256.15805 732...  test/0000.jpg
1  0 0.082653634 716.9591 634.7122 903.4411 1001....  test/0001.jpg
2  0 0.032927833 871.52454 443.7535 1024.0613 641...  test/0002.jpg
3  9 0.8993466 95.215965 228.67722 950.82404 849....  test/0003.jpg
4  0 0.33996734 419.54892 401.3793 661.98517 580....  test/0004.jpg

In [164]:
cfg.test_pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'MultiScaleFlipAug',
  'img_scale': ([128, 128], [384, 384], [640, 640], [960, 960]),
  'flip': True,
  'transforms': [{'type': 'Resize', 'keep_ratio': True},
   {'type': 'RandomFlip'},
   {'type': 'Pad',
    'pad_to_square': True,
    'pad_val': {'img': (114.0, 114.0, 114.0)}},
   {'type': 'DefaultFormatBundle'},
   {'type': 'Collect', 'keys': ['img']}]}]

In [148]:
A = pd.read_csv('work_dirs/inseo_test/yolox_x/submission_TTA_best_bbox_mAP_50_epoch_48.csv')
B = pd.read_csv('work_dirs/inseo_test/yolox_x/submission_best_bbox_mAP_50_epoch_48.csv')

(A != B).sum()

# validation

In [49]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('mmdetection/configs/yolox/yolox_x.py')

root='../dataset/'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + '/kfold/seed41/val_4.json'

#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.pipeline[1]['img_scale'] = (640,640) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [50]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [51]:
epoch = 'best_bbox_mAP_50_epoch_48'

In [52]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
model.eval()

YOLOX(
  (backbone): CSPDarknet(
    (stem): Focus(
      (conv): ConvModule(
        (conv): Conv2d(12, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
    )
    (stage1): Sequential(
      (0): ConvModule(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (activate): Swish()
      )
      (1): CSPLayer(
        (main_conv): ConvModule(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (activate): Swish()
        )
        (short_conv): ConvModule(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(

In [53]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print('number of param :',pytorch_total_params)

number of param : 99004045


In [54]:
checkpoint = torch.load(checkpoint_path)['state_dict']

In [55]:
model.load_state_dict(checkpoint,strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['ema_backbone_stem_conv_conv_weight', 'ema_backbone_stem_conv_bn_weight', 'ema_backbone_stem_conv_bn_bias', 'ema_backbone_stem_conv_bn_running_mean', 'ema_backbone_stem_conv_bn_running_var', 'ema_backbone_stem_conv_bn_num_batches_tracked', 'ema_backbone_stage1_0_conv_weight', 'ema_backbone_stage1_0_bn_weight', 'ema_backbone_stage1_0_bn_bias', 'ema_backbone_stage1_0_bn_running_mean', 'ema_backbone_stage1_0_bn_running_var', 'ema_backbone_stage1_0_bn_num_batches_tracked', 'ema_backbone_stage1_1_main_conv_conv_weight', 'ema_backbone_stage1_1_main_conv_bn_weight', 'ema_backbone_stage1_1_main_conv_bn_bias', 'ema_backbone_stage1_1_main_conv_bn_running_mean', 'ema_backbone_stage1_1_main_conv_bn_running_var', 'ema_backbone_stage1_1_main_conv_bn_num_batches_tracked', 'ema_backbone_stage1_1_short_conv_conv_weight', 'ema_backbone_stage1_1_short_conv_bn_weight', 'ema_backbone_stage1_1_short_conv_bn_bias', 'ema_backbone_stage1_1_short_conv_bn_runni

In [56]:
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [57]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 974/974, 22.7 task/s, elapsed: 43s, ETA:     0s

In [113]:
dir(coco)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'annToMask',
 'annToRLE',
 'anns',
 'catToImgs',
 'cats',
 'createIndex',
 'dataset',
 'download',
 'getAnnIds',
 'getCatIds',
 'getImgIds',
 'imgToAnns',
 'imgs',
 'info',
 'loadAnns',
 'loadCats',
 'loadImgs',
 'loadNumpyAnnotations',
 'loadRes',
 'showAnns']

In [121]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds()[i])[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'val_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,3 0.9199223 238.4238 150.8237 913.77576 696.7233,train/0002.jpg
1,0 0.50894463 350.86237 268.17328 753.71814 829...,train/0006.jpg
2,0 0.4281646 415.1731 220.62004 500.53754 253.0...,train/0015.jpg
3,0 0.09078587 269.8729 354.39124 390.01065 383....,train/0019.jpg
4,0 0.91931367 279.98123 279.7666 715.1434 612.1...,train/0025.jpg
